# Alessandro's scratch notebook

ICESat-2 hackweek final project 
June 16, 2020  
Alessandro Di Bella

Install extra python modules to
* read polygons from WKT strings
* access Sentinel API (we might want to create a virtual environment to avoid doing this every time)

In [1]:
#!pip install wkt
!pip install sentinelsat

In [14]:
### Clear workspace
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [15]:
import sys
### Add path to scripts
if '/home/jovyan/leading_to_phytoplankton/scripts' not in sys.path:
    sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
    
### Import modules
import numpy as np
from sentinelsat import SentinelAPI
import shapely.geometry as sg
from astropy.time import Time, TimeDelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py, pyproj
import readers as rd
from icepyx import icesat2data as ipd
import cartopy.crs as ccrs
import importlib
import search_sentinels as ss
import warnings
warnings.filterwarnings('ignore')

### Magic function to enable interactive plotting
%matplotlib widget 

In [16]:
### Define path to sample data
data_loc = "/home/jovyan/shared/leading_to_phytoplankton/"

In [17]:
# ### Download IS2 sample data
# ## Define search parameters
# short_name = 'ATL03'
spatial_extent = [-110, 70, 70, 90]
# date_range = ['2019-07-26','2019-07-26']
# 
# region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range, \
#                            start_time='21:35:00', end_time='21:36:00', version='002')
# 
# print(region_a.dataset)
# print(region_a.dates)
# print(region_a.start_time)
# print(region_a.end_time)
# print(region_a.dataset_version)
# print(region_a.spatial_extent)
# # region_a.visualize_spatial_extent()
# print(region_a.avail_granules())
# print(region_a.avail_granules(ids=1))



In [18]:
# ## Log in to EarthData
# earthdata_uid = 'ing.a.dibella'
# email = 'ing.a.dibella@gmail.com'
# region_a.earthdata_login(earthdata_uid, email)
# 
# # ## Order granule
# region_a.download_granules(data_loc)

In [19]:
## Load IS2 sample data
#f_atl03 = data_loc + "IS2_S2/ATL03_20190805215948_05930404_002_02.h5"
#f_atl03 = data_loc + "processed_ATL03_20190726213326_04400404_002_01.h5"
f_atl03 = data_loc + "ATL03_20190726213326_04400404_002_01.h5"
f = h5py.File(f_atl03, 'r')
# check to see if it is forward (1)  or backward (0) orientation to know which beam is strong/weak. 
# (2 is transition phase- don't use these data)
print(f['orbit_info/sc_orient'][0])

beam = 'gt2l'
df03 = rd.getATL03(f,beam)
print(df03.head())
print(df03.tail())

print("Number of point measurements:", len(df03))

0
        lats       lons          x          y     heights            dt  conf
0  80.004734  54.366124  8938896.0  42.220333   89.935242  4.941201e+07     0
1  80.004744  54.366109  8938897.0  42.252739 -141.238495  4.941201e+07     0
2  80.004736  54.366121  8938897.0  42.207047  168.690918  4.941201e+07     0
3  80.004743  54.366112  8938897.0  42.228359   16.640863  4.941201e+07     0
4  80.004745  54.366109  8938898.0  42.234760  -29.026579  4.941201e+07     0
               lats        lons           x          y     heights  \
10730602  79.999926 -104.185607  11132819.0  43.878681  -78.278702   
10730603  79.999923 -104.185614  11132820.0  43.835037 -153.670197   
10730604  79.999923 -104.185614  11132820.0  43.833084 -157.039841   
10730605  79.999921 -104.185617  11132820.0  43.810215 -196.544846   
10730606  79.999921 -104.185617  11132820.0  43.815052 -188.187744   

                    dt  conf  
10730602  4.941232e+07     0  
10730603  4.941232e+07     0  
10730604  4.9412

In [20]:
### Downsample ATL03 to speed up test
df03 = df03[::1000]
print("Number of point measurements:", len(df03))

Number of point measurements: 10731


Pull in some extra ATL03 params and convert GPS to UTC time

In [21]:
### Convert GPS time to UTC time
epoch = f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df03['time'] = Time(epoch + df03['dt'],format='gps').utc.datetime

### Calculate along track distance relative to the beginning of the cut segment
df03['AT_dist'] = df03.x - df03.x.values[0]

### Read orbit number
df03['orbit_number'] = [f['/orbit_info/orbit_number'][0]] * len(df03)
df03.head()

,lats,lons,x,y,heights,dt,conf,time,AT_dist,orbit_number
0,80.004734,54.366124,8938896.0,42.220333,89.935242,4.941201e+07,0,2019-07-26 21:33:25.986627,0.0,4802
1000,80.007749,54.362400,8939241.0,41.681538,12.426127,4.941201e+07,0,2019-07-26 21:33:26.035227,345.0,4802
2000,80.008849,54.361042,8939366.0,41.479172,546.315491,4.941201e+07,0,2019-07-26 21:33:26.053327,470.0,4802
3000,80.009568,54.360142,8939448.0,41.564560,750.957275,4.941201e+07,0,2019-07-26 21:33:26.065027,552.0,4802
4000,80.010348,54.359147,8939537.0,42.044792,439.323090,4.941201e+07,0,2019-07-26 21:33:26.077426,641.0,4802


In [22]:
### Plot data
var = 'heights' #choose which variable we want to plot
vmin = -10
vmax = 30
ticks = np.arange(-20,100,5)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
# plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'], df03['lats'], c=df03[var], s=1, cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
importlib.reload(ss)

<module 'search_sentinels' from '/home/jovyan/leading_to_phytoplankton/scripts/search_sentinels.py'>

In [32]:
ss.search_sentinels?

Signature:
ss.search_sentinels(
    platform_name,
    df,
    aoi,
    dt=2,
    user=None,
    pwd=None,
    proj_string='+init=EPSG:3995',
    product_type=None,
    min_cloud_cover=0,
    max_cloud_cover=100,
    swath_type=None,
    f_out=None,
)
Docstring:
Search Sentinel-1/2 images overlapping ICESat-2 data within +- dt

Parameters: (to be finished!)
-----------
platform_name : str ['Sentinel-1 | Sentinel-2']
    name of the platform for which images will be searched
df : panda dataframe
    ICESat-2 data
aoi: str, list
    area of interest as WKT string or bounding box[lllon, lllat, urlon, urlat]
dt: int, float
    difference in hours between CS2 and S2
user : str
    username to connect to the Copernicus Scientific Hub
pwd : str
    password to connect to the Copernicus Scientific Hub
proj_string: str
    projection string to be used with the pyproj module
product_type : str
    name of the type of product to be searched (more info at https://scihub.copernicus.eu/userguide/)
s

In [31]:
out_s1 = ss.search_sentinels('Sentinel-1', df03, spatial_extent, dt=2, user='adia', pwd='ImageSearchTool2020')

Creating AOI polygon...
product_type set to:  GRD
swath_type set to:  EW
Selecting orbit data inside AOI...
N. of orbits/points inside AOI: 1/10731
Query for metadata...


0it [00:00, ?it/s]

N. of total images: 8
Converting time to astropy objects...
Looping over orbits to find intersections within 2h...


1it [00:02,  2.58s/it]
1it [00:00, 94.11it/s]

N. of images within 2h: 6
Creating images footprint polygons...
Looping over orbits to find intersections...
N. of total intersections: 0


In [30]:
out_s2 = ss.search_sentinels('Sentinel-2', df03, spatial_extent, dt=1, user='adia', pwd='ImageSearchTool2020')

Creating AOI polygon...
product_type set to:  S2MSI1C
Selecting orbit data inside AOI...
N. of orbits/points inside AOI: 1/10731
Query for metadata...


Querying products: 100%|██████████| 189/189 [00:01<00:00, 181.69 products/s]
0it [00:00, ?it/s]

N. of total images: 189
Converting time to astropy objects...
Looping over orbits to find intersections within 1h...


1it [00:02,  2.66s/it]
1it [00:00, 28.06it/s]

N. of images within 1h: 189
Creating images footprint polygons...
Looping over orbits to find intersections...
N. of total intersections: 13


In [34]:
out_s2

([4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802,
  4802],
 ['S2A_MSIL1C_20190726T220101_N0208_R072_T13XEK_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T13XEL_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T13XEM_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T13XEJ_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T14XMP_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T14XMQ_20190726T234136.SAFE',
  'S2A_MSIL1C_20190726T220101_N0208_R072_T14XMR_20190726T234136.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T13XEK_20190726T212235.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T13XEJ_20190726T212235.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T13XEL_20190726T212235.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T14XMP_20190726T212235.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T14XMQ_20190726T212235.SAFE',
  'S2B_MSIL1C_20190726T211039_N0208_R143_T1